# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # There is a checkpoint folder ''.ipynb_checkpoints' in my workspace. So I need to skip it.
    if 'ipynb_checkpoints' in root:
        continue
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("sparkifydb")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### **Query 1**:  Give me the **artist**, **song title** and **song's length** in the music app history that was heard during
### **sessionId = 338**, and **itemInSession = 4**
Because we are querying using sessionId and itemInSession as filter, we will partition the table according to sessionId and itemInSession. Therefore they are the primary key and clustering column.

In [17]:
## Create table for query 1
try:
    session.execute("""CREATE TABLE IF NOT EXISTS songs_by_sessionId_and_itemInSession
     (sessionId int, itemInSession int, song text, artist text, length float, PRIMARY KEY (sessionId, itemInSession))""")
except Exception as e:
    print(e)

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_sessionId_and_itemInSession (sessionId, itemInSession, song, artist, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[9], line[0], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

try:
    rows = session.execute("""SELECT artist, song, length FROM songs_by_sessionId_and_itemInSession 
    WHERE sessionId = 338 AND itemInSession = 4
    """)
    column_labels = ['artist', 'song', 'length']
    data = rows.current_rows
    res = pd.DataFrame(data=data, columns=column_labels)
#     for row in rows:
#         print(row.artist, row.song, row.length)
    print(res)
except Exception as e:
    print(e)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### **Query 2**: Give me only the following: **name of artist**, **song (sorted by itemInSession)** and **user (first and last name)**
### for **userid = 10**, **sessionid = 182**
Because we are querying using userId and sessionId as filter, we will partition the table according to userId and sessionId. Therefore they are the primary key and clustering column.

In [20]:
## Create table for query 2
try:
    session.execute("""CREATE TABLE IF NOT EXISTS songs_by_userId_and_sessionId
     (userId int, sessionId int, artist text, song text, user text,
      itemInSession int, PRIMARY KEY (userId, sessionId, itemInSession))""")
except Exception as e:
    print(e)

In [21]:
## Insert data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_userId_and_sessionId (userId, sessionId, artist, song, user, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1]+' '+line[4], int(line[3])))

In [22]:
## SELECT statement to verify the data was entered into the table

try:
    rows = session.execute("""SELECT artist, song, user FROM songs_by_userId_and_sessionId
     WHERE userId = 10 AND sessionId = 182
     ORDER BY sessionId, itemInSession""")
    column_labels = ['artist', 'song', 'user']
    res = pd.DataFrame(data=rows.current_rows, columns=column_labels)
    print(res)
#     for row in rows:
#         print((row))
except Exception as e:
    print(e)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

          user  
0  Sylvie Cruz  
1  Sylvie Cruz  
2  Sylvie Cruz  
3  Sylvie Cruz  


### **Query 3**: Give me every **user name (first and last)** in my music app history who listened to the **song 'All Hands Against His Own'**
Because we are querying using song name as filter, we will partition the table according to song name. Therefore it is the primary key.

In [23]:
## Create a table for query 3.
try:
    session.execute("""CREATE TABLE IF NOT EXISTS users_by_song
     (song text, user text, PRIMARY KEY (song))""")
except Exception as e:
    print(e)
                    

In [24]:
# Insert data into table.
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
    ## TO-DO: Assign the INSERT statements into the `query` variable
            query = "INSERT INTO users_by_song (song, user)"
            query = query + " VALUES (%s, %s)"
            ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
            ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
            session.execute(query, (line[9], line[1]+' '+line[4]))
except Exception as e:
    print(e)

In [27]:
try:
    rows = session.execute("""SELECT user FROM users_by_song WHERE song='All Hands Against His Own' LIMIT 5;""")
    col_label = ['user']
    result3 = pd.DataFrame(data=rows.current_rows, columns=col_label)
    print(result3)
except Exception as e:
    print(e)

           user
0  Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [28]:
session.execute("DROP TABLE IF EXISTS songs_by_sessionId_and_itemInSession")
session.execute("DROP TABLE IF EXISTS songs_by_userId_and_sessionId")
session.execute("DROP TABLE IF EXISTS users_by_song")

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()